# How to transform data into factors

Based on a conceptual understanding of key factor categories, their rationale and popular metrics, a key task is to identify new factors that may better capture the risks embodied by the return drivers laid out previously, or to find new ones. 

In either case, it will be important to compare the performance of innovative factors to that of known factors to identify incremental signal gains.

We create the dataset here and store it in our [data](../data) folder to facilitate reuse in later chapters.

## Imports & Settings

In [6]:
import warnings
warnings.filterwarnings('ignore')

In [7]:
%matplotlib inline

from datetime import datetime
import pandas as pd
import pandas_datareader.data as web
from pprint import pprint


# replaces pyfinance.ols.PandasRollingOLS (no longer maintained)
from statsmodels.regression.rolling import RollingOLS
import statsmodels.api as sm

import matplotlib.pyplot as plt
import seaborn as sns

In [8]:
sns.set_style('whitegrid')
idx = pd.IndexSlice

## Get Data

The `assets.h5` store can be generated using the the notebook [create_datasets](../data/create_datasets.ipynb) in the [data](../data) directory in the root directory of this repo for instruction to download the following dataset.

We load the Quandl stock price datasets covering the US equity markets 2000-18 using `pd.IndexSlice` to perform a slice operation on the `pd.MultiIndex`, select the adjusted close price and unpivot the column to convert the DataFrame to wide format with tickers in the columns and timestamps in the rows:

Set data store location:

In [9]:
from codeload.moonshot.features_store import (
    # add_fundamental_features,
    # add_quality_features,
    # add_price_and_volume_features,
    # add_technical_indicator_features,
    add_securities_master_features,
    # add_market_features
)

In [10]:
from quantrocket import get_prices
qr_prices = get_prices("usstock-1d", 
    start_date="2023-08-01", 
    end_date="2023-09-01", 
    # fields=["Close", "Volume"]  
    fields=["Close"]    
)
closes = qr_prices.loc["Close"]
closes.head()

Sid,FIBBG0000018G2,FIBBG000001LS0,FIBBG000001NT5,FIBBG000001NV2,FIBBG000001R98,FIBBG000001RB5,FIBBG000001RC4,FIBBG000001RD3,FIBBG000001RF1,FIBBG000001RG0,...,QI000000256350,QI000000256351,QI000000256352,QI000000257469,QI000000257707,QI000000257713,QI000000258683,QI000000259639,QI000000259966,QI000000259967
Date,,,,,,,,,,,,,,,,,,,,,
2023-08-01,NaN,25.050,86.9781,86.9593,21.4314,18.3001,16.78,16.89,16.900,16.1300,...,24.5220,26.0391,27.76,0.0436,11.97,1.70,49.8984,NaN,NaN,NaN
2023-08-02,NaN,25.000,78.0826,87.1176,21.5996,18.5788,16.77,NaN,16.900,16.1916,...,24.4620,25.6300,27.40,NaN,12.43,1.70,49.8884,NaN,NaN,NaN
2023-08-03,88.851,25.030,NaN,87.1275,21.0300,NaN,16.75,16.80,16.600,NaN,...,24.3671,25.7000,27.37,0.0305,12.11,NaN,49.8685,NaN,NaN,NaN
2023-08-04,88.693,25.020,NaN,87.1968,21.2000,18.4203,16.60,16.68,16.803,16.1500,...,24.4520,25.5597,27.27,0.0463,11.72,1.55,49.8934,NaN,NaN,NaN
2023-08-07,NaN,25.025,76.0267,87.3651,21.2000,18.4000,16.76,16.67,NaN,NaN,...,24.4220,25.6000,27.38,0.0453,11.39,NaN,49.8984,NaN,NaN,NaN


In [11]:
# securities_master_features = add_securities_master_features(qr_prices)
# pprint(list(securities_master_features.keys()))

In [12]:
# fundamental_features = add_fundamental_features(qr_prices)
# pprint(list(fundamental_features.keys()))


In [13]:
# quality_features = add_quality_features(qr_prices)
# pprint(list(quality_features.keys()))


In [14]:
# price_and_volume_features = add_price_and_volume_features(qr_prices)
# pprint(list(price_and_volume_features.keys()))


In [15]:
# technical_indicator_features = add_technical_indicator_features(qr_prices)
# pprint(list(technical_indicator_features.keys()))


In [16]:
# market_features = add_market_features(qr_prices)
# pprint(list(market_features.keys()))


In [17]:
DATA_STORE = '/codeload/machine-learning-for-trading/data/assets.h5'


In [18]:
# START = 2000
# END = 2018

In [19]:
with pd.HDFStore(DATA_STORE) as store:
    print(store.info())
    # prices = (store['quandl/wiki/prices']
    #           .loc[idx[str(START):str(END), :], 'adj_close']
    #           .unstack('ticker'))
    stocks = store['us_equities/stocks'].loc[:, ['marketcap', 'ipoyear', 'sector']]
# stocks

<class 'pandas.io.pytables.HDFStore'>
File path: /codeload/machine-learning-for-trading/data/assets.h5
/engineered_features            frame        (shape->[144733,33])  
/qr/usstock-1d                  frame        (shape->[21050,24074])
/quandl/wiki/prices             frame        (shape->[15389314,12])
/quandl/wiki/stocks             frame        (shape->[1,2])        
/sp500/stocks                   frame        (shape->[503,6])      
/us_equities/stocks             frame        (shape->[3678,3])     


### Keep data with stock info

Remove `stocks` duplicates and align index names for later joining.

In [20]:
from quantrocket.master import get_securities_reindexed_like
securities = get_securities_reindexed_like(closes, fields=["Exchange", "Symbol"])

In [21]:
prices = closes.copy()
symbols = securities.loc["Symbol"]
prices.columns = symbols.iloc[0].tolist()
prices.columns.name = 'ticker'
prices.index.name = 'date'

In [22]:
duplicated_columns = prices.columns[prices.columns.duplicated()]
prices = prices.drop(columns=duplicated_columns)

stocks = stocks[~stocks.index.duplicated()]
stocks.index.name = 'ticker'
shared = prices.columns.intersection(stocks.index)

In [23]:
stocks = stocks.loc[shared, :]
stocks.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3448 entries, MPU to HSPOW
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   marketcap  3448 non-null   float64
 1   ipoyear    3448 non-null   float64
 2   sector     3448 non-null   object 
dtypes: float64(2), object(1)
memory usage: 107.8+ KB


In [24]:
prices = prices.loc[:, shared]

prices.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 24 entries, 2023-08-01 to 2023-09-01
Columns: 3448 entries, MPU to HSPOW
dtypes: float64(3448)
memory usage: 646.7 KB


In [25]:
assert prices.shape[1] == stocks.shape[0]


Get tickers with both price information and metdata

In [26]:
prices

ticker,MPU,DMF,AAPL,CET,CNSL,CLM,NATH,CGEN,RCS,CPT,...,AUROW,BYTSW,DBGIW,SLDPW,BEAT,MOBBW,JFBRW,NXLIW,PLTNW,HSPOW
date,,,,,,,,,,,,,,,,,,,,,
2023-08-01,1.380,6.3388,195.3412,37.1100,3.63,8.4709,79.4373,1.19,5.3208,108.41,...,0.560000,0.221000,NaN,0.470000,3.040,NaN,NaN,NaN,NaN,NaN
2023-08-02,1.360,6.3189,192.3203,36.6450,3.58,8.3835,80.2120,1.15,5.3699,107.19,...,0.570000,NaN,NaN,0.439599,2.900,0.300000,NaN,NaN,0.0500,0.0648
2023-08-03,1.370,6.1995,190.9122,36.8300,3.55,8.3738,78.6229,1.12,5.3797,106.49,...,0.540100,0.205100,NaN,0.448400,3.020,NaN,NaN,0.1100,0.0509,NaN
2023-08-04,1.360,6.2294,181.7446,36.9300,3.58,8.4612,77.5601,1.12,5.5074,108.54,...,0.559899,0.266600,4.55,0.420101,2.880,0.280000,NaN,NaN,0.0400,NaN
2023-08-07,1.340,6.1796,178.6088,37.0499,3.66,8.4709,78.0965,1.03,5.4877,109.90,...,0.491400,NaN,4.98,0.393500,2.770,0.274899,NaN,NaN,0.0400,NaN
2023-08-08,1.320,6.1896,179.5575,36.7000,3.90,8.4612,77.6396,1.05,5.4976,108.62,...,0.539999,NaN,NaN,0.390100,2.770,0.275000,NaN,NaN,0.0300,NaN
2023-08-09,1.340,6.2045,177.9497,36.9200,3.97,8.5098,77.5800,1.05,5.4681,108.64,...,0.539900,NaN,5.04,0.400200,2.630,0.275000,0.0684,NaN,0.0362,NaN
2023-08-10,1.350,6.1946,177.7300,36.7600,3.88,8.5389,77.3516,1.07,5.3997,107.70,...,0.529899,0.254100,NaN,0.400000,2.640,NaN,NaN,NaN,0.0362,NaN
2023-08-11,1.330,6.2245,177.7900,36.8600,3.98,8.4806,77.4708,1.11,5.3601,107.86,...,0.560000,NaN,NaN,0.387535,2.575,0.275000,NaN,0.0640,0.0350,NaN


## Create monthly return series

To reduce training time and experiment with strategies for longer time horizons, we convert the business-daily data to month-end frequency using the available adjusted close price:

In [ ]:
monthly_prices = prices.resample('M').last()

To capture time series dynamics that reflect, for example, momentum patterns, we compute historical returns using the method `.pct_change(n_periods)`, that is, returns over various monthly periods as identified by lags.

We then convert the wide result back to long format with the `.stack()` method, use `.pipe()` to apply the `.clip()` method to the resulting `DataFrame`, and winsorize returns at the [1%, 99%] levels; that is, we cap outliers at these percentiles.

Finally, we normalize returns using the geometric average. After using `.swaplevel()` to change the order of the `MultiIndex` levels, we obtain compounded monthly returns for six periods ranging from 1 to 12 months:

In [ ]:
monthly_prices

In [ ]:
monthly_prices.info()

In [28]:
outlier_cutoff = 0.01
data = pd.DataFrame()
lags = [1, 2, 3, 6, 9, 12]
for lag in lags:
    data[f'return_{lag}m'] = (prices
                           .pct_change(lag)
                           .stack()
                           .pipe(lambda x: x.clip(lower=x.quantile(outlier_cutoff),
                                                  upper=x.quantile(1-outlier_cutoff)))
                           .add(1)
                           .pow(1/lag)
                           .sub(1)
                           )
data = data.swaplevel().dropna()
data.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 40567 entries, ('MPU', Timestamp('2023-08-17 00:00:00')) to ('HSPOW', Timestamp('2023-09-01 00:00:00'))
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   return_1m   40567 non-null  float64
 1   return_2m   40567 non-null  float64
 2   return_3m   40567 non-null  float64
 3   return_6m   40567 non-null  float64
 4   return_9m   40567 non-null  float64
 5   return_12m  40567 non-null  float64
dtypes: float64(6)
memory usage: 2.1+ MB


## Drop stocks with less than 10 yrs of returns

In [ ]:
min_obs = 100
nobs = data.groupby(level='ticker').size()
keep = nobs[nobs>min_obs].index

data = data.loc[idx[keep,:], :]
data.info()

In [ ]:
data.describe()

In [ ]:
# cmap = sns.diverging_palette(10, 220, as_cmap=True)
sns.clustermap(data.corr('spearman'), annot=True, center=0, cmap='Blues');

We are left with 1,670 tickers.

In [ ]:
data.index.get_level_values('ticker').nunique()

## Rolling Factor Betas

We will introduce the Fama—French data to estimate the exposure of assets to common risk factors using linear regression in [Chapter 9, Time Series Models](../09_time_series_models).

The five Fama—French factors, namely market risk, size, value, operating profitability, and investment have been shown empirically to explain asset returns and are commonly used to assess the risk/return profile of portfolios. Hence, it is natural to include past factor exposures as financial features in models that aim to predict future returns.

We can access the historical factor returns using the `pandas-datareader` and estimate historical exposures using the `RollingOLS` rolling linear regression functionality in the `statsmodels` library as follows:

Use Fama-French research factors to estimate the factor exposures of the stock in the dataset to the 5 factors market risk, size, value, operating profitability and investment.

In [ ]:
factors = ['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA']
factor_data = web.DataReader('F-F_Research_Data_5_Factors_2x3', 'famafrench', start='2000')[0].drop('RF', axis=1)
factor_data.index = factor_data.index.to_timestamp()
factor_data = factor_data.resample('M').last().div(100)
factor_data.index.name = 'date'
factor_data.info()

In [ ]:
data

In [ ]:
factor_data = factor_data.join(data['return_1m']).sort_index()
factor_data.info()

In [ ]:
T = 24
betas = (factor_data.groupby(level='ticker',
                             group_keys=False)
         .apply(lambda x: RollingOLS(endog=x.return_1m,
                                     exog=sm.add_constant(x.drop('return_1m', axis=1)),
                                     window=min(T, x.shape[0]-1))
                .fit(params_only=True)
                .params
                .drop('const', axis=1)))

In [ ]:
betas.describe().join(betas.sum(1).describe().to_frame('total'))

In [ ]:
cmap = sns.diverging_palette(10, 220, as_cmap=True)
sns.clustermap(betas.corr(), annot=True, cmap=cmap, center=0);

In [ ]:
data = (data
        .join(betas
              .groupby(level='ticker')
              .shift()))
data.info()

### Impute mean for missing factor betas

In [ ]:
data.loc[:, factors] = data.groupby('ticker')[factors].apply(lambda x: x.fillna(x.mean()))
data.info()

## Momentum factors

We can use these results to compute momentum factors based on the difference between returns over longer periods and the most recent monthly return, as well as for the difference between 3 and 12 month returns as follows:

In [ ]:
for lag in [2,3,6,9,12]:
    data[f'momentum_{lag}'] = data[f'return_{lag}m'].sub(data.return_1m)
data[f'momentum_3_12'] = data[f'return_12m'].sub(data.return_3m)

## Date Indicators

In [ ]:
dates = data.index.get_level_values('date')
data['year'] = dates.year
data['month'] = dates.month

## Lagged returns

To use lagged values as input variables or features associated with the current observations, we use the .shift() method to move historical returns up to the current period:

In [ ]:
for t in range(1, 7):
    data[f'return_1m_t-{t}'] = data.groupby(level='ticker').return_1m.shift(t)
data.info()

## Target: Holding Period Returns

Similarly, to compute returns for various holding periods, we use the normalized period returns computed previously and shift them back to align them with the current financial features

In [ ]:
for t in [1,2,3,6,12]:
    data[f'target_{t}m'] = data.groupby(level='ticker')[f'return_{t}m'].shift(-t)

In [ ]:
cols = ['target_1m',
        'target_2m',
        'target_3m', 
        'return_1m',
        'return_2m',
        'return_3m',
        'return_1m_t-1',
        'return_1m_t-2',
        'return_1m_t-3']

data[cols].dropna().sort_index().head(10)

In [ ]:
data.info()

## Create age proxy

We use quintiles of IPO year as a proxy for company age.

In [ ]:
data = (data
        .join(pd.qcut(stocks.ipoyear, q=5, labels=list(range(1, 6)))
              .astype(float)
              .fillna(0)
              .astype(int)
              .to_frame('age')))
data.age = data.age.fillna(-1)

## Create dynamic size proxy

We use the marketcap information from the NASDAQ ticker info to create a size proxy.

In [ ]:
stocks.info()

Market cap information is tied to currrent prices. We create an adjustment factor to have the values reflect lower historical prices for each individual stock:

In [ ]:
size_factor = (monthly_prices
               .loc[data.index.get_level_values('date').unique(),
                    data.index.get_level_values('ticker').unique()]
               .sort_index(ascending=False)
               .pct_change()
               .fillna(0)
               .add(1)
               .cumprod())
size_factor.info()

In [ ]:
msize = (size_factor
         .mul(stocks
              .loc[size_factor.columns, 'marketcap'])).dropna(axis=1, how='all')

### Create Size indicator as deciles per period

Compute size deciles per month:

In [ ]:
data['msize'] = (msize
                 .apply(lambda x: pd.qcut(x, q=10, labels=list(range(1, 11)))
                        .astype(int), axis=1)
                 .stack()
                 .swaplevel())
data.msize = data.msize.fillna(-1)

## Combine data

In [ ]:
data = data.join(stocks[['sector']])
data.sector = data.sector.fillna('Unknown')

In [ ]:
data.info()

## Store data

We will use the data again in several later chapters, starting in [Chapter 7 on Linear Models](../07_linear_models).

In [ ]:
with pd.HDFStore(DATA_STORE) as store:
    store.put('engineered_features', data.sort_index())
    print(store.info())

## Create Dummy variables

For most models, we need to encode categorical variables as 'dummies' (one-hot encoding):

In [ ]:
dummy_data = pd.get_dummies(data,
                            columns=['year','month', 'msize', 'age',  'sector'],
                            prefix=['year','month', 'msize', 'age', ''],
                            prefix_sep=['_', '_', '_', '_', ''])
dummy_data = dummy_data.rename(columns={c:c.replace('.0', '') for c in dummy_data.columns})
dummy_data.info()